<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/docs/examples/finetuning/embeddings/finetune_embedding_adapter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="在 Colab 中打开"/></a>


# 在任何黑盒嵌入模型之上微调适配器

在 LlamaIndex 中，我们具有能力允许您在任何模型（sentence_transformers、OpenAI 等）产生的嵌入之上微调适配器。

这使您能够将您的嵌入表示转换为一个针对您特定数据和查询进行优化的新潜在空间。这可能会导致检索性能的小幅提升，从而进一步提高 RAG 系统的性能。

我们通过我们的 `EmbeddingAdapterFinetuneEngine` 抽象来实现这一点。我们微调三种类型的适配器：
- 线性
- 2 层神经网络
- 自定义神经网络


## 生成语料库

我们使用我们的辅助抽象`generate_qa_embedding_pairs`来生成我们的训练和评估数据集。这个函数接受任何文本节点（块）并生成一个包含（问题，上下文）对的结构化数据集。


In [ ]:
%pip install llama-index-embeddings-openai
%pip install llama-index-embeddings-adapter
%pip install llama-index-finetuning

In [ ]:
import json

from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.schema import MetadataMode

# 下载数据


In [ ]:
!mkdir -p 'data/10k/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/uber_2021.pdf' -O 'data/10k/uber_2021.pdf'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/lyft_2021.pdf' -O 'data/10k/lyft_2021.pdf'

In [ ]:
TRAIN_FILES = ["./data/10k/lyft_2021.pdf"]
VAL_FILES = ["./data/10k/uber_2021.pdf"]

TRAIN_CORPUS_FPATH = "./data/train_corpus.json"
VAL_CORPUS_FPATH = "./data/val_corpus.json"

In [ ]:
def load_corpus(files, verbose=False):
    if verbose:
        print(f"Loading files {files}")

    reader = SimpleDirectoryReader(input_files=files)
    docs = reader.load_data()
    if verbose:
        print(f"Loaded {len(docs)} docs")

    parser = SentenceSplitter()
    nodes = parser.get_nodes_from_documents(docs, show_progress=verbose)

    if verbose:
        print(f"Parsed {len(nodes)} nodes")

    return nodes

我们通过将Lyft语料库作为训练数据集，将Uber语料库作为验证数据集，进行非常简单的训练/验证分割。


In [ ]:
train_nodes = load_corpus(TRAIN_FILES, verbose=True)
val_nodes = load_corpus(VAL_FILES, verbose=True)

Loading files ['../../../examples/data/10k/lyft_2021.pdf']
Loaded 238 docs


Parsing documents into nodes:   0%|          | 0/238 [00:00<?, ?it/s]

Parsed 349 nodes
Loading files ['../../../examples/data/10k/uber_2021.pdf']
Loaded 307 docs


Parsing documents into nodes:   0%|          | 0/307 [00:00<?, ?it/s]

Parsed 418 nodes


### 生成合成查询

现在，我们使用一个LLM（gpt-3.5-turbo）来使用语料库中的每个文本块作为上下文生成问题。

每个（生成的问题，作为上下文使用的文本块）成为微调数据集中的一个数据点（用于训练或评估）。


In [ ]:
from llama_index.finetuning import generate_qa_embedding_pairs
from llama_index.core.evaluation import EmbeddingQAFinetuneDataset

In [ ]:
train_dataset = generate_qa_embedding_pairs(train_nodes)
val_dataset = generate_qa_embedding_pairs(val_nodes)

train_dataset.save_json("train_dataset.json")
val_dataset.save_json("val_dataset.json")

In [ ]:
# [可选] 加载
train_dataset = EmbeddingQAFinetuneDataset.from_json("train_dataset.json")
val_dataset = EmbeddingQAFinetuneDataset.from_json("val_dataset.json")

## 运行嵌入微调

然后，我们在现有嵌入模型的基础上对线性适配器进行微调。我们导入新的`EmbeddingAdapterFinetuneEngine`抽象，该抽象接受现有的嵌入模型和一组训练参数。


#### 调整 bge-small-en 模型（默认）


In [ ]:
from llama_index.finetuning import EmbeddingAdapterFinetuneEngine
from llama_index.core.embeddings import resolve_embed_model
import torch

base_embed_model = resolve_embed_model("local:BAAI/bge-small-en")

finetune_engine = EmbeddingAdapterFinetuneEngine(
    train_dataset,
    base_embed_model,
    model_output_path="model_output_test",
    # bias=True,
    epochs=4,
    verbose=True,
    # optimizer_class=torch.optim.SGD,
    # optimizer_params={"lr": 0.01}
)

In [ ]:
finetune_engine.finetune()

In [ ]:
embed_model = finetune_engine.get_finetuned_model()

# 或者导入模型
from llama_index.core.embeddings import LinearAdapterEmbeddingModel

# embed_model = LinearAdapterEmbeddingModel(base_embed_model, "model_output_test")

## 评估微调模型

我们将微调模型与基础模型以及text-embedding-ada-002进行比较。

我们使用两种排名指标进行评估：
- **命中率指标**：对于每个（查询，上下文）对，我们检索具有该查询的前k个文档。如果结果包含真实的上下文，则命中。
- **平均倒数排名**：这是一种稍微更精细的排名指标，它查看在前k个检索集中真实上下文的“倒数排名”。倒数排名定义为1/排名。当然，如果结果不包含上下文，则倒数排名为0。


In [ ]:
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import VectorStoreIndex
from llama_index.core.schema import TextNode
from tqdm.notebook import tqdm
import pandas as pd

from eval_utils import evaluate, display_results

In [ ]:
ada = OpenAIEmbedding()
ada_val_results = evaluate(val_dataset, ada)

Generating embeddings:   0%|          | 0/395 [00:00<?, ?it/s]

100%|████████████████████████████████████████████████████████████████| 790/790 [03:03<00:00,  4.30it/s]


In [ ]:
display_results(["ada"], [ada_val_results])

,retrievers,hit_rate,mrr
0,ada,0.870886,0.72884


In [ ]:
bge = "local:BAAI/bge-small-en"
bge_val_results = evaluate(val_dataset, bge)

Generating embeddings:   0%|          | 0/395 [00:00<?, ?it/s]

100%|████████████████████████████████████████████████████████████████| 790/790 [00:23<00:00, 33.76it/s]


In [ ]:
display_results(["bge"], [bge_val_results])

,retrievers,hit_rate,mrr
0,bge,0.787342,0.643038


In [ ]:
ft_val_results = evaluate(val_dataset, embed_model)

Generating embeddings:   0%|          | 0/395 [00:00<?, ?it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 790/790 [00:21<00:00, 36.95it/s]


In [ ]:
display_results(["ft"], [ft_val_results])

,retrievers,hit_rate,mrr
0,ft,0.798734,0.662152


这里我们展示所有的结果连接在一起。


In [ ]:
display_results(
    ["ada", "bge", "ft"], [ada_val_results, bge_val_results, ft_val_results]
)

,retrievers,hit_rate,mrr
0,ada,0.870886,0.730105
1,bge,0.787342,0.643038
2,ft,0.798734,0.662152


## 微调双层适配器

让我们也尝试微调一个双层神经网络！

这是一个简单的双层神经网络，其中包含一个ReLU激活函数和一个残差层。

我们进行25个epochs的训练 - 比线性适配器更长 - 并且每100步保留一个检查点。


In [ ]:
# 需要torch依赖
来自llama_index.core.embeddings.adapter_utils的TwoLayerNN

来自llama_index.finetuning的EmbeddingAdapterFinetuneEngine

来自llama_index.core.embeddings的resolve_embed_model

来自llama_index.embeddings.adapter的AdapterEmbeddingModel

In [ ]:
base_embed_model = resolve_embed_model("local:BAAI/bge-small-en")
adapter_model = TwoLayerNN(
    384,  # 输入维度
    1024,  # 隐藏维度
    384,  # 输出维度
    bias=True,  # 是否使用偏置
    add_residual=True,  # 是否添加残差连接
)

finetune_engine = EmbeddingAdapterFinetuneEngine(
    train_dataset,  # 训练数据集
    base_embed_model,  # 基础嵌入模型
    model_output_path="model5_output_test",  # 模型输出路径
    model_checkpoint_path="model5_ck",  # 模型检查点路径
    adapter_model=adapter_model,  # 适配器模型
    epochs=25,  # 迭代次数
    verbose=True,  # 是否显示详细信息
)

In [ ]:
finetune_engine.finetune()

In [ ]:
embed_model_2layer = finetune_engine.get_finetuned_model(
    adapter_cls=TwoLayerNN
)

### 评估结果

运行与上一节中使用的相同的评估脚本，以测量两层模型中的命中率/平均倒数排名（MRR）。


In [ ]:
# 从中间的检查点加载模型
embed_model_2layer = AdapterEmbeddingModel(
    base_embed_model,
    "model5_output_test",
    TwoLayerNN,
)

In [ ]:
from eval_utils import evaluate, display_results

In [ ]:
ft_val_results_2layer = evaluate(val_dataset, embed_model_2layer)

Generating embeddings:   0%|          | 0/395 [00:00<?, ?it/s]

100%|████████████████████████████████████████████████████████████████| 790/790 [00:21<00:00, 36.93it/s]


In [ ]:
# 如果你还没有运行ada/bge，请将以下内容注释掉
display_results(
    ["ada", "bge", "ft_2layer"],
    [ada_val_results, bge_val_results, ft_val_results_2layer],
)

# 如果你只想显示微调模型的结果，请取消注释以下内容
# display_results(["ft_2layer"], [ft_val_results_2layer])

,retrievers,hit_rate,mrr
0,ada,0.870886,0.728840
1,bge,0.787342,0.643038
2,ft_2layer,0.798734,0.662848


In [ ]:
# 从中间的检查点加载模型
embed_model_2layer_s900 = AdapterEmbeddingModel(
    base_embed_model,
    "model5_ck/step_900",
    TwoLayerNN,
)

In [ ]:
ft_val_results_2layer_s900 = evaluate(val_dataset, embed_model_2layer_s900)

Generating embeddings:   0%|          | 0/395 [00:00<?, ?it/s]

100%|████████████████████████████████████████████████████████████████| 790/790 [00:19<00:00, 40.57it/s]


In [ ]:
# 如果你还没有运行ada/bge，请将以下内容注释掉
display_results(
    ["ada", "bge", "ft_2layer_s900"],
    [ada_val_results, bge_val_results, ft_val_results_2layer_s900],
)

# 如果你只想显示微调模型的结果，请取消注释以下内容
# display_results(["ft_2layer_s900"], [ft_val_results_2layer_s900])

,retrievers,hit_rate,mrr
0,ada,0.870886,0.728840
1,bge,0.787342,0.643038
2,ft_2layer_s900,0.803797,0.667426


## 尝试自定义模型

您可以在这里定义自己的自定义适配器！只需对`BaseAdapter`进行子类化，它是`nn.Module`类的一个轻量级包装器。

您只需要对`forward`和`get_config_dict`进行子类化。

只需确保您熟悉编写`PyTorch`代码 :)


In [ ]:
from llama_index.core.embeddings.adapter_utils import BaseAdapter
import torch.nn.functional as F
from torch import nn, Tensor
from typing import Dict

In [ ]:
class CustomNN(BaseAdapter):
    """自定义神经网络转换。

    这是我们的TwoLayerNN的一个副本，这里展示出来是为了笔记本的目的。

    参数:
        in_features (int): 输入维度。
        hidden_features (int): 隐藏层维度。
        out_features (int): 输出维度。
        bias (bool): 是否使用偏置。默认为False。
        activation_fn_str (str): 激活函数的名称。默认为"relu"。

    """

    def __init__(
        self,
        in_features: int,
        hidden_features: int,
        out_features: int,
        bias: bool = False,
        add_residual: bool = False,
    ) -> None:
        super(CustomNN, self).__init__()
        self.in_features = in_features
        self.hidden_features = hidden_features
        self.out_features = out_features
        self.bias = bias

        self.linear1 = nn.Linear(in_features, hidden_features, bias=True)
        self.linear2 = nn.Linear(hidden_features, out_features, bias=True)
        self._add_residual = add_residual
        # 如果添加残差，则添加残差权重（初始化为0）
        self.residual_weight = nn.Parameter(torch.zeros(1))

    def forward(self, embed: Tensor) -> Tensor:
        """前向传播（Wv）。

        参数:
            embed (Tensor): 输入张量。

        """
        output1 = self.linear1(embed)
        output1 = F.relu(output1)
        output2 = self.linear2(output1)

        if self._add_residual:
            output2 = self.residual_weight * output2 + embed

        return output2

    def get_config_dict(self) -> Dict:
        """获取配置字典。"""
        return {
            "in_features": self.in_features,
            "hidden_features": self.hidden_features,
            "out_features": self.out_features,
            "bias": self.bias,
            "add_residual": self._add_residual,
        }

In [ ]:
custom_adapter = CustomNN(
    384,  # 输入维度
    1024,  # 隐藏维度
    384,  # 输出维度
    bias=True,  # 是否使用偏置
    add_residual=True,  # 是否添加残差连接
)

finetune_engine = EmbeddingAdapterFinetuneEngine(
    train_dataset,
    base_embed_model,
    model_output_path="custom_model_output",
    model_checkpoint_path="custom_model_ck",
    adapter_model=custom_adapter,
    epochs=25,  # 迭代次数
    verbose=True,  # 是否显示详细信息
)


In [ ]:
finetune_engine.finetune()

In [ ]:
embed_model_custom = finetune_engine.get_finetuned_model(
    adapter_cls=CustomAdapter
)

### 评估结果

运行与上一节中使用的相同的评估脚本，以测量命中率/平均倒数排名（MRR）。


In [ ]:
# [可选] 手动加载模型
# embed_model_custom = AdapterEmbeddingModel(
#     base_embed_model,
#     "custom_model_ck/step_300",
#     TwoLayerNN,
# )

In [ ]:
from eval_utils import evaluate, display_results

In [ ]:
ft_val_results_custom = evaluate(val_dataset, embed_model_custom)

Generating embeddings:   0%|          | 0/395 [00:00<?, ?it/s]

100%|████████████████████████████████████████████████████████████████| 790/790 [00:20<00:00, 37.77it/s]


In [ ]:
display_results(["ft_custom"]x, [ft_val_results_custom])

,retrievers,hit_rate,mrr
0,ft_custom,0.789873,0.645127
